### Load Images

In [2]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


2.0.0


#### Constant Variables

In [1]:
# Pfad zum Hauptordner, der die Unterordner mit Bildern enthält
main_folder = 'path_to_main_folder'

# Auflösung der Bilder in das Format für das neuronale Netz
desired_height = 256
desired_width = 256

# Anzahl der zu klassifizierenden Schilder inklusive "kein Schild"
number_of_classes = 8

#### Data Augmentation Pipeline

In [3]:
# Erstellen eines ImageDataGenerator für Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,   # Normalisieren der Bilder
    rotation_range=40,  # Zufällige Rotation der Bilder
    width_shift_range=0.2,  # Horizontaler Shift
    height_shift_range=0.2, # Vertikaler Shift
    shear_range=0.2,        # Scherung
    zoom_range=0.2,         # Zoom
    horizontal_flip=True,   # Horizontales Flippen
    fill_mode='nearest'     # Füllmethode für neue Pixel
)

#### Load images with tf.flow_from_directory() to directly create corresponding labels 

In [4]:
# Laden der Bilder und Labels aus den Unterordnern
train_generator = train_datagen.flow_from_directory(
    main_folder,
    target_size=(desired_height, desired_width),  # Zielgröße der Bilder
    batch_size=32,
    class_mode='categorical'  # Für mehrklassige Klassifizierung
)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\aaron\anaconda3\envs\deep_learning\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\aaron\AppData\Local\Temp\ipykernel_13336\2889319893.py", line 9, in <module>
    class_mode='categorical'  # Für mehrklassige Klassifizierung
  File "c:\Users\aaron\anaconda3\envs\deep_learning\lib\site-packages\keras_preprocessing\image\image_data_generator.py", line 543, in flow_from_directory
    dtype=self.dtype
  File "c:\Users\aaron\anaconda3\envs\deep_learning\lib\site-packages\keras_preprocessing\image\directory_iterator.py", line 115, in __init__
    for subdir in sorted(os.listdir(directory)):
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path_to_main_folder'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\aaron\anaconda3\envs\deep_learning\lib

#### Create own model

In [ ]:
# Erstellen des Modells (hier ein einfaches Beispiel)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(desired_height, desired_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Weitere Schichten nach Bedarf hinzufügen
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(number_of_classes, activation='softmax')  # Anzahl der Klassen
])

#### Compile and train model

In [ ]:
# Kompilieren des Modells
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training des Modells
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=15,
    # Weitere Parameter nach Bedarf
)